In [ ]:
https://claude.ai/chat/0b610bfa-55cd-465f-90f6-995b3d1714cc

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118
  Obtaining dependency information for torch from https://download.pytorch.org/whl/cu118/torch-2.7.0%2Bcu118-cp311-cp311-win_amd64.whl.metadata
  Using cached https://download.pytorch.org/whl/cu118/torch-2.7.0%2Bcu118-cp311-cp311-win_amd64.whl.metadata (29 kB)
  Obtaining dependency information for torchvision from https://download.pytorch.org/whl/cu118/torchvision-0.22.0%2Bcu118-cp311-cp311-win_amd64.whl.metadata
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.22.0%2Bcu118-cp311-cp311-win_amd64.whl.metadata (6.3 kB)
  Obtaining dependency information for torchaudio from https://download.pytorch.org/whl/cu118/torchaudio-2.7.0%2Bcu118-cp311-cp311-win_amd64.whl.metadata
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.7.0%2Bcu118-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
  Obtaining dependency information for typi

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
                                              0.0/375.9 MB ? eta -:--:--
                                            0.0/375.9 MB 991.0 kB/s eta 0:06:20
                                              0.2/375.9 MB 1.6 MB/s eta 0:03:50
                                              0.4/375.9 MB 2.5 MB/s eta 0:02:28
                                              0.8/375.9 MB 4.3 MB/s eta 0:01:28
                                              1.7/375.9 MB 7.1 MB/s eta 0:00:54
                                             3.4/375.9 MB 12.1 MB/s eta 0:00:31
                                             5.9/375.9 MB 17.1 MB/s eta 0:00:22
     -                                       9.8/375.9 MB 24.9 MB/s eta 0:00:15
     -                                      12.1/375.9 MB 65.6 MB/s eta 0:00:06
     -                                      12.3/375.9 MB 50.4 MB/s eta 0:00:08
     -                                      12.6/

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
transformers 2.1.1 requires sentencepiece, which is not installed.
numba 0.57.0 requires numpy<1.25,>=1.21, but you have numpy 2.1.3 which is incompatible.
scipy 1.10.1 requires numpy<1.27.0,>=1.19.5, but you have numpy 2.1.3 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import torch
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import pickle
from tqdm import tqdm

In [ ]:
import torch
import os

# Configuration
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EMB_MODEL_NAME = "deepseek-ai/deepseek-coder-1.3b-base"
BATCH_SIZE = 8
MAX_LENGTH = 512
CACHE_DIR = "embeddings_cache"
MODEL_DIR = "models"

# Create folders if they don't exist
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

print(f"Using device: {DEVICE}")

Using device: cuda


In [ ]:
!pip install transformers accelerate

In [ ]:
import torch
print(torch.cuda.is_available())  # Should be True
print(torch.cuda.get_device_name(0))

True
RTX A4000


In [ ]:
# Create directories if they don't exist
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

In [ ]:
file_path = "C:/Users/g738x956/Wsdm_Dataset/train.parquet"

In [ ]:
!pip install pyarrow

In [ ]:
# Read the Parquet file
df = pd.read_parquet(file_path)

# Show basic info
print(df.shape)
print(df.columns)
df.head()

(48439, 8)
Index(['id', 'prompt', 'response_a', 'response_b', 'winner', 'model_a',
       'model_b', 'language'],
      dtype='object')


,id,prompt,response_a,response_b,winner,model_a,model_b,language
0,00007cff95d7f7974642a785aca248b0f26e60d3312fac...,vieš po Slovensky?,"Áno, hovorím po slovensky. Ako vám môžem pomôcť?","Áno, veď som tu! Môžem ti pomôcť s otázkami al...",model_a,o1-preview,reka-core-20240904,Slovak
1,00010ed04b536f56ebe43eef1100c13906abea12bf9855...,You will be given a piece of news. Analyze it ...,Let's break down the news and analyze it accor...,"```json\n{\n ""contains_orgs"": true,\n ""orgs""...",model_a,gemma-2-27b-it,gemini-1.5-flash-002,Russian
2,0003800d510e38803efba5ceaec122bc66408fe367b0be...,"Dört basamaklı, rakamları birbirinden ve sıfır...","Bu soruyu çözmek için, verilen koşulları adım ...",Bu problemi adım adım çözelim:\n\n1) ABCD - DC...,model_a,gpt-4-0125-preview,claude-3-5-sonnet-20240620,Turkish
3,00072026c68f5418ef2da238394e418ce72a534b9b22d5...,"현재 추천된 탑 3 종목인 Cabaletta Bio (CABA), Rocket Ph...","죄송하지만 저는 금융 조언을 제공할 수 없습니다. 저는 AI 모델이며, 투자 결정에...",현재 추천된 탑 3 종목에 순위를 매기기 위해서는 여러 가지 요소들을 고려해야 합니...,model_b,gemma-2-2b-it,llama-3.1-nemotron-70b-instruct,English
4,0007ce7cf6bc1b5a8f8a4669b854fb12030863c970d9dc...,Please be boring,"Alright, I'll be as boring as possible.\n\nTod...","Understood. Here is a straightforward, unadorn...",model_a,reka-flash-20240722,grok-2-2024-08-13,English


In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
from transformers import AutoTokenizer, AutoModelForCausalLM
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Memory optimization for PyTorch
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
# Configuration
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EMB_MODEL_NAME = "deepseek-ai/deepseek-coder-1.3b-base"
BATCH_SIZE = 16
MAX_LENGTH = 512
CACHE_DIR = "embeddings_cache"
MODEL_DIR = "models"

# Create directories if they don't exist
os.makedirs(CACHE_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

In [ ]:

# --------------------------------------------------
# 1. Data Preparation
# --------------------------------------------------
def prepare_data(train_path, sample_fraction=0.1, english_only=True):
    """Load and prepare Parquet dataset file, filtering for English language data."""
    print(f"Loading data from {train_path}...")

    # Load train parquet file
    df_train_full = pd.read_parquet(train_path)

    print("Full train dataset shape:", df_train_full.shape)

    # Display column information
    print("\nTrain columns:", df_train_full.columns.tolist())
    print("Sample data from train:")
    print(df_train_full.head(2))

    # Check the unique values in the language column
    if 'language' in df_train_full.columns:
        print("\nUnique values in language column:")
        print(df_train_full['language'].value_counts())

    # Handle missing values
    df_train_full = df_train_full.dropna()

    # Filter for English language using the language column
    if english_only and 'language' in df_train_full.columns:
        print("\nFiltering for English language data only...")
        # Check if 'en' or 'English' is used in the dataset
        if 'English' in df_train_full['language'].unique():
            english_data = df_train_full[df_train_full['language'] == 'English']
        elif 'english' in df_train_full['language'].unique():
            english_data = df_train_full[df_train_full['language'] == 'english']
        elif 'en' in df_train_full['language'].unique():
            english_data = df_train_full[df_train_full['language'] == 'en']
        else:
            print("WARNING: Could not find 'English', 'english' or 'en' in language column.")
            print("Available language values:", df_train_full['language'].unique())
            english_data = df_train_full  # Keep all data if we can't identify English

        print(f"English data shape: {english_data.shape} (from original {df_train_full.shape})")
        df_train_full = english_data
    elif english_only:
        print("\nWARNING: 'language' column not found in dataset. Cannot filter for English only.")

    # Sample a fraction of the data
    sampled_data = df_train_full.sample(frac=sample_fraction, random_state=42)
    print(f"\nSampled {sample_fraction*100}% of data, shape:", sampled_data.shape)

    return sampled_data

# --------------------------------------------------
# 2. Embedding Generation with DeepSeek
# --------------------------------------------------
def setup_embedding_model(model_name=EMB_MODEL_NAME):
    """Set up tokenizer and model for embedding generation."""
    print(f"Loading embedding model: {model_name}")
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    # Load in half precision to save memory
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16
    ).to(DEVICE)
    return tokenizer, model

def get_embeddings(texts, tokenizer, model):
    """Generate embeddings for a batch of texts."""
    inputs = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=MAX_LENGTH,
        return_tensors="pt"
    ).to(DEVICE)

    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)

    # Mean pooling from last hidden state
    embeddings = outputs.hidden_states[-1].mean(dim=1).cpu().numpy()
    return embeddings

def batch_generate_embeddings(text_series, tokenizer, model, cache_file=None, smaller_batch_size=4):
    """Generate embeddings in batches with optional caching and memory management."""
    # Check if cache exists
    if cache_file and os.path.exists(cache_file):
        print(f"Loading cached embeddings from {cache_file}")
        with open(cache_file, 'rb') as f:
            return pickle.load(f)

    print(f"Generating embeddings for {len(text_series)} texts...")
    embeddings = []

    for i in tqdm(range(0, len(text_series), BATCH_SIZE)):
        batch = text_series.iloc[i:i+BATCH_SIZE].tolist()

        # Process in even smaller sub-batches to manage memory
        sub_embeddings = []
        for j in range(0, len(batch), smaller_batch_size):
            sub_batch = batch[j:j+smaller_batch_size]
            # Clear cache before each sub-batch
            torch.cuda.empty_cache()
            sub_embeddings.append(get_embeddings(sub_batch, tokenizer, model))

        embeddings.append(np.vstack(sub_embeddings))

        # Periodically clear cache
        if i % (BATCH_SIZE * 10) == 0:
            torch.cuda.empty_cache()

    result = np.vstack(embeddings)

    # Cache embeddings
    if cache_file:
        print(f"Caching embeddings to {cache_file}")
        with open(cache_file, 'wb') as f:
            pickle.dump(result, f)

    return result

# --------------------------------------------------
# 3. Feature Engineering
# --------------------------------------------------
def create_features(prompt_embs, resp_a_embs, resp_b_embs):
    """Create feature vectors from embeddings."""
    print("Creating feature vectors...")
    # Concatenate prompt with responses
    feature_a = np.concatenate([prompt_embs, resp_a_embs], axis=1)
    feature_b = np.concatenate([prompt_embs, resp_b_embs], axis=1)

    # Create difference features (abs difference captures semantic distinction)
    X = np.abs(feature_a - feature_b)

    # Optional: Add more features like cosine similarity
    cosine_sim_a = np.sum(prompt_embs * resp_a_embs, axis=1) / (np.linalg.norm(prompt_embs, axis=1) * np.linalg.norm(resp_a_embs, axis=1))
    cosine_sim_b = np.sum(prompt_embs * resp_b_embs, axis=1) / (np.linalg.norm(prompt_embs, axis=1) * np.linalg.norm(resp_b_embs, axis=1))
    sim_diff = np.abs(cosine_sim_a - cosine_sim_b).reshape(-1, 1)
    X = np.hstack([X, sim_diff])

    return X

# --------------------------------------------------
# 4. Model Training
# --------------------------------------------------
def build_model(input_dim, dropout_rate=0.3):
    """Build and compile neural network classifier with Functional API."""
    # Using Functional API instead of Sequential to avoid input_shape warning
    inputs = tf.keras.layers.Input(shape=(input_dim,))
    x = tf.keras.layers.Dense(256, activation='relu')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_rate / 2)(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC()]
    )

    return model

def train_model(X_train, y_train, X_val, y_val, input_dim):
    """Train the model with callbacks and class balancing."""
    print("Building and training model...")

    # Print shapes before training to verify they match
    print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
    print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")

    model = build_model(input_dim)

    # Calculate class weights
    class_counts = np.bincount(y_train)
    class_weights = {i: len(y_train) / (len(np.unique(y_train)) * count) for i, count in enumerate(class_counts)}

    callbacks = [
        EarlyStopping(monitor='val_auc', patience=5, mode='max', restore_best_weights=True),
        ModelCheckpoint(os.path.join(MODEL_DIR, "best_model.keras"),
                      monitor='val_auc', mode='max', save_best_only=True)
    ]

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=30,
        batch_size=16,  # Increased batch size to speed up training
        class_weight=class_weights,
        callbacks=callbacks,
        verbose=1
    )

    return model, history

# --------------------------------------------------
# 5. Evaluation
# --------------------------------------------------
def evaluate_model(model, X, y, label="Test"):
    """Evaluate model performance with multiple metrics."""
    print(f"\n{label} Set Evaluation:")
    y_pred_prob = model.predict(X, verbose=0)
    y_pred = (y_pred_prob > 0.5).astype(int).flatten()

    metrics = {
        'accuracy': accuracy_score(y, y_pred),
        'f1': f1_score(y, y_pred),
        'roc_auc': roc_auc_score(y, y_pred_prob)
    }

    print(pd.Series(metrics))
    print("\nClassification Report:")
    print(classification_report(y, y_pred))

    return metrics

# --------------------------------------------------
# 6. Make Test Predictions
# --------------------------------------------------
def generate_submission(model, tokenizer, embedding_model, df_test, submission_path="submission.csv"):
    """Generate predictions for test set and save submission file."""
    print("Generating test predictions...")

    # Generate embeddings for test data
    test_prompt_embs = batch_generate_embeddings(
        df_test['prompt'],
        tokenizer,
        embedding_model,
        cache_file=os.path.join(CACHE_DIR, "test_prompt_embs.pkl")
    )

    test_resp_a_embs = batch_generate_embeddings(
        df_test['response_a'],
        tokenizer,
        embedding_model,
        cache_file=os.path.join(CACHE_DIR, "test_resp_a_embs.pkl")
    )

    test_resp_b_embs = batch_generate_embeddings(
        df_test['response_b'],
        tokenizer,
        embedding_model,
        cache_file=os.path.join(CACHE_DIR, "test_resp_b_embs.pkl")
    )

    # Create features for test data
    X_test = create_features(test_prompt_embs, test_resp_a_embs, test_resp_b_embs)

    # Generate predictions
    y_pred_prob = model.predict(X_test, verbose=0).flatten()

    # Create submission dataframe
    submission = pd.DataFrame()
    submission['id'] = df_test.index
    submission['prediction'] = y_pred_prob

    # Save submission file
    submission.to_csv(submission_path, index=False)
    print(f"Submission saved to {submission_path}")

    return submission

# --------------------------------------------------
# 7. Main Function
# --------------------------------------------------
def main(train_path, sample_fraction=0.1):
    """Main function to run the full pipeline using only a fraction of training data."""
    # Load and prepare data
    df_data = prepare_data(train_path, sample_fraction, english_only=True)

    # Split the sampled data into train, validation, and test sets (70/15/15)
    train_data, temp_data = train_test_split(df_data, test_size=0.3, random_state=42)
    val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

    print(f"Split sizes - Train: {len(train_data)}, Validation: {len(val_data)}, Test: {len(test_data)}")

    # Setup embedding model
    tokenizer, embedding_model = setup_embedding_model()

    # NEW: Single function to generate features AND labels together
    def generate_features_and_labels(dataset, prefix):
        """Generate both features and labels together to ensure proper alignment."""
        print(f"\nGenerating features and labels for {prefix} data...")

        prompt_embs = batch_generate_embeddings(
            dataset['prompt'],
            tokenizer,
            embedding_model,
            cache_file=os.path.join(CACHE_DIR, f"{prefix}_prompt_embs.pkl")
        )

        resp_a_embs = batch_generate_embeddings(
            dataset['response_a'],
            tokenizer,
            embedding_model,
            cache_file=os.path.join(CACHE_DIR, f"{prefix}_resp_a_embs.pkl")
        )

        resp_b_embs = batch_generate_embeddings(
            dataset['response_b'],
            tokenizer,
            embedding_model,
            cache_file=os.path.join(CACHE_DIR, f"{prefix}_resp_b_embs.pkl")
        )

        # Create feature vectors
        X = create_features(prompt_embs, resp_a_embs, resp_b_embs)

        # Create corresponding labels
        def map_winner(winner):
            if winner in ['A', 'model_a']:
                return 1
            elif winner in ['B', 'model_b']:
                return 0
            else:
                print(f"Warning: Unexpected winner value: {winner}")
                return None

        y = dataset['winner'].apply(map_winner).values

        # Verify shapes match
        print(f"{prefix} shapes - X: {X.shape}, y: {y.shape}")

        if len(X) != len(y):
            print(f"WARNING: Shape mismatch in {prefix} set. Trimming to match...")
            min_len = min(len(X), len(y))
            X = X[:min_len]
            y = y[:min_len]
            print(f"After trimming: X: {X.shape}, y: {y.shape}")

        return X, y

    # Generate features and labels for all datasets
    X_train, y_train = generate_features_and_labels(train_data, "train")
    X_val, y_val = generate_features_and_labels(val_data, "val")
    X_test, y_test = generate_features_and_labels(test_data, "test")

    # Train model
    model, history = train_model(X_train, y_train, X_val, y_val, X_train.shape[1])

    # Evaluate on validation set
    val_metrics = evaluate_model(model, X_val, y_val, label="Validation")

    # Evaluate on test set
    test_metrics = evaluate_model(model, X_test, y_test, label="Test")

    return model, tokenizer, embedding_model, val_metrics, test_metrics

if __name__ == "__main__":
    # Replace with your actual file path
    train_path = "C:/Users/g738x956/Wsdm_Dataset/train.parquet"

    # You can adjust the sample fraction as needed
    sample_fraction = 0.9  # Using 50% of data

    # Run the pipeline with English-only data
    model, tokenizer, embedding_model, val_metrics, test_metrics = main(train_path, sample_fraction)

Loading data from C:/Users/g738x956/Wsdm_Dataset/train.parquet...
Full train dataset shape: (48439, 8)

Train columns: ['id', 'prompt', 'response_a', 'response_b', 'winner', 'model_a', 'model_b', 'language']
Sample data from train:
                                                  id  \
0  00007cff95d7f7974642a785aca248b0f26e60d3312fac...   
1  00010ed04b536f56ebe43eef1100c13906abea12bf9855...   

                                              prompt  \
0                                 vieš po Slovensky?   
1  You will be given a piece of news. Analyze it ...   

                                          response_a  \
0   Áno, hovorím po slovensky. Ako vám môžem pomôcť?   
1  Let's break down the news and analyze it accor...   

                                          response_b   winner         model_a  \
0  Áno, veď som tu! Môžem ti pomôcť s otázkami al...  model_a      o1-preview   
1  ```json\n{\n  "contains_orgs": true,\n  "orgs"...  model_a  gemma-2-27b-it   

                m

C:\Users\g738x956\.conda\envs\tf-venv\lib\site-packages\keras\src\callbacks\early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_auc` which is not available. Available metrics are: accuracy,auc_6,loss,val_accuracy,val_auc_6,val_loss
  current = self.get_monitor_value(logs)
C:\Users\g738x956\.conda\envs\tf-venv\lib\site-packages\keras\src\callbacks\model_checkpoint.py:209: UserWarning: Can save best model only with val_auc available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


993/993 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - accuracy: 0.4988 - auc_6: 0.5011 - loss: 0.7065 - val_accuracy: 0.5116 - val_auc_6: 0.5077 - val_loss: 0.7027
Epoch 3/30
993/993 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - accuracy: 0.5026 - auc_6: 0.5016 - loss: 0.7005 - val_accuracy: 0.5078 - val_auc_6: 0.5029 - val_loss: 0.6955
Epoch 4/30
993/993 ━━━━━━━━━━━━━━━━━━━━ 16s 16ms/step - accuracy: 0.4971 - auc_6: 0.5031 - loss: 0.6987 - val_accuracy: 0.4957 - val_auc_6: 0.4963 - val_loss: 0.6938
Epoch 5/30
993/993 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - accuracy: 0.5213 - auc_6: 0.5234 - loss: 0.6931 - val_accuracy: 0.5110 - val_auc_6: 0.5044 - val_loss: 0.6945
Epoch 6/30
993/993 ━━━━━━━━━━━━━━━━━━━━ 16s 16ms/step - accuracy: 0.5175 - auc_6: 0.5294 - loss: 0.6927 - val_accuracy: 0.4972 - val_auc_6: 0.5106 - val_loss: 0.7103
Epoch 7/30
993/993 ━━━━━━━━━━━━━━━━━━━━ 15s 15ms/step - accuracy: 0.5249 - auc_6: 0.5383 - loss: 0.6906 - val_accuracy: 0.5096 - val_auc_6: 0.5117 - val_loss: 0.6966
Epoch 8/30
993/

In [ ]:
# 1. Enhanced feature creation
def create_enhanced_features(prompt_embs, resp_a_embs, resp_b_embs):
    # Original features
    feature_a = np.concatenate([prompt_embs, resp_a_embs], axis=1)
    feature_b = np.concatenate([prompt_embs, resp_b_embs], axis=1)
    X = np.abs(feature_a - feature_b)

    # Add element-wise multiplication features (capture interactions)
    interact_a = prompt_embs * resp_a_embs
    interact_b = prompt_embs * resp_b_embs
    interact_diff = np.abs(interact_a - interact_b)

    # Multiple similarity metrics
    from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

    # Reshape for pairwise metrics
    p_reshaped = prompt_embs.reshape(prompt_embs.shape[0], 1, prompt_embs.shape[1])
    ra_reshaped = resp_a_embs.reshape(resp_a_embs.shape[0], 1, resp_a_embs.shape[1])
    rb_reshaped = resp_b_embs.reshape(resp_b_embs.shape[0], 1, resp_b_embs.shape[1])

    # Cosine similarity (already have this)
    cosine_sim_a = np.sum(prompt_embs * resp_a_embs, axis=1) / (np.linalg.norm(prompt_embs, axis=1) * np.linalg.norm(resp_a_embs, axis=1))
    cosine_sim_b = np.sum(prompt_embs * resp_b_embs, axis=1) / (np.linalg.norm(prompt_embs, axis=1) * np.linalg.norm(resp_b_embs, axis=1))

    # Euclidean distance
    euc_dist_a = np.sqrt(np.sum(np.square(prompt_embs - resp_a_embs), axis=1)).reshape(-1, 1)
    euc_dist_b = np.sqrt(np.sum(np.square(prompt_embs - resp_b_embs), axis=1)).reshape(-1, 1)
    euc_dist_diff = np.abs(euc_dist_a - euc_dist_b)

    # Combine all features
    sim_diff = np.abs(cosine_sim_a - cosine_sim_b).reshape(-1, 1)
    X = np.hstack([X, interact_diff, sim_diff, euc_dist_diff])

    return X

# 2. Improved model architecture
def build_improved_model(input_dim, dropout_rate=0.5):
    inputs = tf.keras.layers.Input(shape=(input_dim,))

    # First block
    x = tf.keras.layers.Dense(512, kernel_regularizer=tf.keras.regularizers.l2(0.001))(inputs)
    x = tf.keras.layers.LeakyReLU(alpha=0.1)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)

    # Second block
    x = tf.keras.layers.Dense(256, kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.1)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)

    # Third block
    x = tf.keras.layers.Dense(128, kernel_regularizer=tf.keras.regularizers.l2(0.001))(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.1)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_rate/2)(x)

    # Output layer
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # Use a lower learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC()]
    )

    return model

# 3. Advanced training with learning rate scheduling
def train_improved_model(X_train, y_train, X_val, y_val, input_dim):
    model = build_improved_model(input_dim)

    # Calculate balanced class weights
    from sklearn.utils.class_weight import compute_class_weight
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

    # Learning rate scheduler
    lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=0.00001,
        verbose=1
    )

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_auc',
            patience=10,
            mode='max',
            restore_best_weights=True,
            min_delta=0.001
        ),
        tf.keras.callbacks.ModelCheckpoint(
            os.path.join(MODEL_DIR, "best_model.keras"),
            monitor='val_auc',
            mode='max',
            save_best_only=True
        ),
        lr_scheduler
    ]

    # Train with more epochs but EarlyStopping will prevent overfitting
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=32,
        class_weight=class_weight_dict,
        callbacks=callbacks,
        verbose=1
    )

    return model, history